# SOC 210B Assignment 4
_By M. Nolan Gray_

In [11]:
# Importing libraries, checking out the dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
dtafile = './Data/nlsw88.dta'
df = pd.read_stata(dtafile)
df.head(5)

,idcode,age,race,married,never_married,grade,collgrad,south,smsa,c_city,industry,occupation,union,wage,hours,ttl_exp,tenure
0,1,37,black,single,0,12.0,not college grad,0,SMSA,0,Transport/Comm/Utility,Operatives,union,11.739125,48.0,10.333334,5.333333
1,2,37,black,single,0,12.0,not college grad,0,SMSA,1,Manufacturing,Craftsmen,union,6.400963,40.0,13.621795,5.250000
2,3,42,black,single,1,12.0,not college grad,0,SMSA,1,Manufacturing,Sales,NaN,5.016723,40.0,17.730770,1.250000
3,4,43,white,married,0,17.0,college grad,0,SMSA,0,Professional Services,Other,union,9.033813,42.0,13.211537,1.750000
4,6,42,white,married,0,12.0,not college grad,0,SMSA,0,Manufacturing,Operatives,nonunion,8.083731,48.0,17.820513,17.750000


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2246 entries, 0 to 2245
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   idcode         2246 non-null   int16   
 1   age            2246 non-null   int8    
 2   race           2246 non-null   category
 3   married        2246 non-null   category
 4   never_married  2246 non-null   int8    
 5   grade          2244 non-null   float64 
 6   collgrad       2246 non-null   category
 7   south          2246 non-null   int8    
 8   smsa           2246 non-null   category
 9   c_city         2246 non-null   int8    
 10  industry       2232 non-null   category
 11  occupation     2237 non-null   category
 12  union          1878 non-null   category
 13  wage           2246 non-null   float32 
 14  hours          2242 non-null   float64 
 15  ttl_exp        2246 non-null   float32 
 16  tenure         2231 non-null   float32 
dtypes: category(7), float32(3), float

## Question 1

In [4]:
# Counts for categorical variables
for col in df.select_dtypes(['category']):
    print(df[col].value_counts(), "\n") 

white    1637
black     583
other      26
Name: race, dtype: int64 

married    1442
single      804
Name: married, dtype: int64 

not college grad    1714
college grad         532
Name: collgrad, dtype: int64 

SMSA       1581
nonSMSA     665
Name: smsa, dtype: int64 

Professional Services      824
Manufacturing              367
Wholesale/Retail Trade     333
Finance/Ins/Real Estate    192
Public Administration      176
Personal Services           97
Transport/Comm/Utility      90
Business/Repair Svc         86
Construction                29
Ag/Forestry/Fisheries       17
Entertainment/Rec Svc       17
Mining                       4
Name: industry, dtype: int64 

Sales                     726
Professional/technical    317
Laborers                  286
Managers/admin            264
Operatives                246
Other                     187
Clerical/unskilled        102
Craftsmen                  53
Transport                  28
Service                    16
Farm laborers             

In [5]:
# Continuous variable means and standard deviations
for col in df.select_dtypes(['float64','float32']):
    print(df[col].name,"Mean: ",df[col].mean(), "Standard deviation: ", df[col].std(), "\n") 

grade Mean:  13.098930481283423 Standard deviation:  2.5212460945811084 

wage Mean:  7.766931056976318 Standard deviation:  5.755528450012207 

hours Mean:  37.218108831400535 Standard deviation:  10.509135117595548 

ttl_exp Mean:  12.534981727600098 Standard deviation:  4.610212326049805 

tenure Mean:  5.977848529815674 Standard deviation:  5.510325908660889 



In [6]:
# I probably could have written a fancier loop that included
# age w/o all those pesky dummy variables, but I'm a dummy myself
print(df['age'].name,"Mean: ",df[col].mean(), "Standard deviation: ", df[col].std(), "\n") 

age Mean:  5.977848529815674 Standard deviation:  5.510325908660889 



## Question 2

In [7]:
# These are the explanatory variables
X = df[['wage','collgrad','union','ttl_exp','race']]

In [8]:
# Race needs to be turned into a dummy; 
# you will notice White is the reference category.
X = pd.get_dummies(data=X, drop_first=True)
X.shape

(2246, 6)

In [9]:
X.sample(5)

,wage,ttl_exp,collgrad_college grad,union_union,race_black,race_other
731,3.019324,6.160257,1,0,0,0
561,3.344482,14.339743,0,0,0,0
930,5.032206,16.685896,0,0,0,0
1636,7.938803,14.391026,0,0,1,0
1757,7.745568,7.961538,1,1,0,0


In [14]:
X = X.rename(columns = {'wage':'wage','ttl_exp':'ttl_exp', 'collgrad_college grad':'coll_dummy', 'union_union':'union_dummy','race_black':'black','race_other':'other'})

In [15]:
model_interaction = smf.ols(formula='wage ~ ttl_exp + coll_dummy + union_dummy + black + other', data=X).fit()
summary = model_interaction.summary()
summary.tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,3.4530,0.336,10.281,0.000,2.794,4.112
ttl_exp,0.2980,0.025,12.051,0.000,0.250,0.347
coll_dummy,3.1028,0.270,11.500,0.000,2.574,3.632
union_dummy,0.6381,0.283,2.254,0.024,0.083,1.193
black,-1.1126,0.260,-4.275,0.000,-1.623,-0.602
other,0.0724,1.059,0.068,0.945,-2.005,2.150


So the regression formula is *y = 3.45 + 0.30(x0) + 3.10(x1) + 0.64(x2) - 1.11(x3) + 0.07(x4), where:
* x0 = ttl_exp
* x1 = collgrad
* x2 = union
* x3 = race_black
* x4 = race_other (Not significant!)

So the formula for a black woman who is a unionized college graduate with 10 years of experience would have a predicted wage of:
* 3.45 + 0.30(10) + 3.10(1) + 0.64(1) - 1.11(1) + 0.07(0) = *$9.08*

Her older sister with 20 years of experience would have a predicted wage of:
* 3.45 + 0.30(20) + 3.10(1) + 0.64(1) - 1.11(1) + 0.07(0) = *$12.08*

## Question 3

In [ ]:
# First, let's test to make sure the output is consistent
model_interaction = smf.ols(formula='wage ~ X', data=df).fit()
summary = model_interaction.summary()
summary.tables[1]

In [ ]:
# It is. Let's do some cleaning.
X.head(5)

In [ ]:
model_interaction = smf.ols(formula='wage ~ ttl_exp + coll_dummy + union_dummy + black + other', data=X).fit()
summary = model_interaction.summary()
summary.tables[1]

In [ ]:
X.head(5)

In [ ]:
X["idcode"] = X.index + 1
X['idcode'] = X['idcode'].astype(np.int16)

In [ ]:
df.info()

In [ ]:
X.info()

In [ ]:
merged = pd.merge(df, X, on="idcode",how='right')
merged.shape

In [ ]:
merged.sample(10)

In [ ]:
print(df.iloc[1429])